### 1) Imports

In [2]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Imports scr info
customers_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_customers_dataset.csv')).rename(columns={'customer_zip_code_prefix': 'zip_code_prefix'}).drop(['customer_unique_id', 'customer_city'], axis=1)
geolocation_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_geolocation_dataset.csv')).rename(columns={'geolocation_zip_code_prefix': 'zip_code_prefix'}).drop(['geolocation_city', 
                                                                                                                                            'geolocation_state'], axis=1)
order_items_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_order_items_dataset.csv')).drop(['order_item_id', 'seller_id','shipping_limit_date'], axis=1)
order_payments_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_order_payments_dataset.csv')).drop(['payment_sequential', 'payment_installments'], axis=1)
order_reviews_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_order_reviews_dataset.csv')).groupby(by=['order_id'], as_index=False)['review_score'].mean()
orders_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_orders_dataset.csv')).drop(['order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1)
products_dataset = pd.read_csv(os.path.join('..', 'src', 'olist_products_dataset.csv')).drop(['product_name_lenght','product_description_lenght', 
        'product_photos_qty', 'product_weight_g','product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1)
product_category_name_translation = pd.read_csv(os.path.join('..', 'src', 'product_category_name_translation.csv'))

### 2) Treatments

In [33]:
# Merge datasets

## Customers_Info
customers_info = pd.merge(customers_dataset, geolocation_dataset, on='zip_code_prefix', how='inner').drop('zip_code_prefix', axis=1)
customers_info = customers_info.iloc[customers_info.groupby('customer_id')['geolocation_lat'].agg(pd.Series.idxmax)]
customers_info = customers_info.query('geolocation_lat <= 5.27438888' and 'geolocation_lng >= -73.98283055' and 'geolocation_lat >= -33.75116944' and 'geolocation_lng <=  -34.79314722')

## Purchases_Info

purchases_info = pd.merge(order_items_dataset, products_dataset, on='product_id', how='inner')
purchases_info = pd.merge(purchases_info, order_payments_dataset, on='order_id', how='inner')
purchases_info = pd.merge(purchases_info, orders_dataset, on='order_id', how='inner').query("product_category_name in ('cama_mesa_banho', 'beleza_saude', 'esporte_lazer')")
purchases_info = pd.merge(purchases_info, product_category_name_translation, on='product_category_name', how='inner').drop(['product_category_name'], axis=1)[['customer_id', 'order_id', 
                                                                                                                                                                        'product_id', 'product_category_name_english', 'price', 'freight_value', 'payment_type', 'payment_value']]
purchases_info = pd.merge(purchases_info, order_reviews_dataset, on='order_id', how='inner').drop(['order_id'], axis=1)

In [36]:
# Aggregate datasets and rename columns
purchases_df = purchases_info.pivot_table(index='customer_id',columns='product_category_name_english',values=['payment_value', 'review_score'], fill_value=0)
purchases_df.columns = [f'{j}_{i}' for i,j in purchases_df.columns]
purchases_df.reset_index(inplace=True)

products = purchases_info.pivot_table(index='customer_id',columns='product_category_name_english',values='product_id', aggfunc= 'count', fill_value=0)
products.columns = [f'qty_of_products_{i}' for i in products.columns]
products.reset_index(inplace=True)

# Merge final data
from functools import reduce

dfs = [customers_info, products, purchases_df]

customers_purchases_and_reviews = reduce(lambda  left,right: pd.merge(left,right,on=['customer_id'], how='inner'), dfs).query('qty_of_products_bed_bath_table > 2 or qty_of_products_health_beauty > 2 or qty_of_products_sports_leisure > 2')

# Fuction to create regions

region = {
    
    'AC': 'north',
    'AP': 'north',
    'AM': 'north',
    'PA': 'north',
    'RO': 'north',
    'RR': 'north',
    'TO': 'north',
    'AL': 'northeast',
    'PB': 'northeast',
    'BA': 'northeast',
    'CE': 'northeast',
    'MA': 'northeast',
    'PE': 'northeast',
    'PI': 'northeast',
    'RN': 'northeast',
    'SE': 'northeast',
    'DF': 'mid-west',
    'GO': 'mid-west',
    'MT': 'mid-west',
    'MS': 'mid-west',
    'ES': 'southeast',
    'MG': 'southeast',
    'RJ': 'southeast',
    'SP': 'southeast',
    'PR': 'South',
    'RS': 'South',
    'SC': 'South'
   
}
customers_purchases_and_reviews['customers_region'] = customers_purchases_and_reviews['customer_state'].map(region).fillna(-1)

customers_purchases_and_reviews = customers_purchases_and_reviews[['customer_id', 'customer_state','customers_region', 'geolocation_lat', 'geolocation_lng',
                    'qty_of_products_bed_bath_table','bed_bath_table_payment_value','bed_bath_table_review_score',
                    'qty_of_products_health_beauty', 'health_beauty_payment_value', 'health_beauty_review_score',
                    'qty_of_products_sports_leisure', 'sports_leisure_payment_value','sports_leisure_review_score']]

# 500 customers sample                       
customers_purchases_and_reviews = customers_purchases_and_reviews.sample(500)

In [37]:
customers_purchases_and_reviews

,customer_id,customer_state,customers_region,geolocation_lat,geolocation_lng,qty_of_products_bed_bath_table,bed_bath_table_payment_value,bed_bath_table_review_score,qty_of_products_health_beauty,health_beauty_payment_value,health_beauty_review_score,qty_of_products_sports_leisure,sports_leisure_payment_value,sports_leisure_review_score
4223,2939201dbb94797337b7e580fb86cd32,AM,north,-3.11,-60.00,0,0.00,0.00,4,260.60,1.00,0,0.00,0.00
11522,7313faf1758192058cf30aaaba45cfe0,MG,southeast,-19.92,-43.95,0,0.00,0.00,0,0.00,0.00,4,256.00,1.00
18425,b741c151088f770a5189db10c519e937,MA,northeast,-5.17,-47.79,0,0.00,0.00,4,528.80,3.00,0,0.00,0.00
720,06ec7aed3161859018db02f695b6600d,SP,southeast,-24.00,-46.41,4,98.72,2.00,0,0.00,0.00,0,0.00,0.00
13504,86b212924fe60bc925c0a8f0b0b14d87,MG,southeast,-19.86,-43.91,3,266.25,1.00,0,0.00,0.00,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19346,c0573fb95d387c7d26ee75592bd66991,BA,northeast,-13.01,-38.48,0,0.00,0.00,0,0.00,0.00,3,42.40,4.00
16075,a06e9a7060cee74dbde1a2d5d7117747,SP,southeast,-23.27,-46.69,0,0.00,0.00,0,0.00,0.00,3,84.64,2.00
24608,f5498d05cf105bdd3bc601ff77601d7b,SP,southeast,-23.52,-46.22,0,0.00,0.00,7,30.37,4.00,0,0.00,0.00
8605,55840ff4caf1d80ac296d0b1ac968b53,SP,southeast,-22.75,-47.36,5,216.45,4.00,0,0.00,0.00,0,0.00,0.00


In [29]:
# save dataset 

outname = 'customers_purchases_reviews_olist.csv'

outdir = '../data_treated'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

customers_purchases_and_reviews.to_csv(fullname)